In [1]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import NewWhaleGenerator,create_newWhale_model
from utility.grayscale_DenseNet import DenseNet_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import DenseNet121

In [2]:
color = False
shapes = (224,224,1)
HalfBatch = 8

### set up generator

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=1)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

gen_train = NewWhaleGenerator(Ids_train.Imgs.tolist(),newWhale_train,transform,HalfBatch=HalfBatch)
gen_val = NewWhaleGenerator(Ids_val.Imgs.tolist(),newWhale_val,transform_test,HalfBatch=HalfBatch)

### build model

In [5]:
conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
#conv_base = Xception_greyscale((256,256,1),'max',False)
# conv_base = Xception_reduced((256,256,1))

In [6]:
train_model = create_newWhale_model(1e-3,conv_base)

### training

In [7]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model2(1e-3,margin_loss_fun_factory(0.1,5),conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
625/625 [==============================] - 38s 61ms/step - loss: 0.7845 - acc: 0.5793 - val_loss: 0.8333 - val_acc: 0.5154
Epoch 2/5
625/625 [==============================] - 30s 48ms/step - loss: 0.6624 - acc: 0.6086 - val_loss: 0.7121 - val_acc: 0.5597
Epoch 3/5
625/625 [==============================] - 30s 49ms/step - loss: 0.6569 - acc: 0.6137 - val_loss: 0.7334 - val_acc: 0.5588
Epoch 4/5
625/625 [==============================] - 30s 49ms/step - loss: 0.6523 - acc: 0.6194 - val_loss: 0.7499 - val_acc: 0.5737
Epoch 5/5
625/625 [==============================] - 30s 49ms/step - loss: 0.6485 - acc: 0.6242 - val_loss: 0.7386 - val_acc: 0.5648
time:160.0806107521057


Fine-Tune

In [8]:
train_model.layers[1].trainable=True

In [9]:
train = False
for layer in train_model.layers[1].layers:
    if 'conv5' in layer.name:
        train = True
    layer.trainable = train

In [10]:
train_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=4e-4))

In [11]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
625/625 [==============================] - 51s 82ms/step - loss: 0.6472 - val_loss: 0.6828
Epoch 2/5
625/625 [==============================] - 40s 64ms/step - loss: 0.6311 - val_loss: 0.7210
Epoch 3/5
625/625 [==============================] - 40s 64ms/step - loss: 0.6231 - val_loss: 0.6238
Epoch 4/5
625/625 [==============================] - 40s 64ms/step - loss: 0.6148 - val_loss: 0.6714
Epoch 5/5
625/625 [==============================] - 40s 64ms/step - loss: 0.6045 - val_loss: 0.6273
time:216.07224869728088


In [12]:
train_model.layers[1].trainable=True

In [13]:
train = False
for layer in train_model.layers[1].layers:
    if 'conv4' in layer.name:
        train = True
    layer.trainable = train

In [14]:
train_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=1e-4),metrics=['acc'])

In [15]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
625/625 [==============================] - 79s 126ms/step - loss: 0.5986 - acc: 0.6770 - val_loss: 0.6584 - val_acc: 0.6325
Epoch 2/5
625/625 [==============================] - 65s 104ms/step - loss: 0.5701 - acc: 0.7036 - val_loss: 0.6614 - val_acc: 0.6152
Epoch 3/5
625/625 [==============================] - 81s 130ms/step - loss: 0.5570 - acc: 0.7079 - val_loss: 0.6296 - val_acc: 0.6483
Epoch 4/5
625/625 [==============================] - 106s 170ms/step - loss: 0.5341 - acc: 0.7342 - val_loss: 0.6311 - val_acc: 0.6465
Epoch 5/5
625/625 [==============================] - 106s 169ms/step - loss: 0.5173 - acc: 0.7411 - val_loss: 0.6290 - val_acc: 0.6604
time:446.42661333084106


In [16]:
train_model.save('Models/NewWhale.h5')